In [1]:
from typing import Final
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

%matplotlib inline

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_excel('historyIndex.xls')
df = df.iloc[6:647].copy()

In [4]:
df.reset_index(drop=True, inplace=True)
df.columns = ['date', 'price']

In [5]:
df['date'] = pd.to_datetime(df['date'])
df = df.replace(',','', regex=True)
df['price'] = df['price'].astype(float)

In [6]:
periods = ['1m', '3m', '6m', '1y', '2y', '3y', '5y', '10y', '20y']
durations = [1, 3, 6, 12, 24, 36, 60, 120, 240]

In [7]:
for period, duration in zip(periods, durations):
    df[f'{period}_cumulative'] = df['price'].pct_change(periods=duration)

In [8]:
for period, duration in zip(periods, durations):
    df[f'{period}_annualized'] = (1 + df[f'{period}_cumulative'])**(12/duration) - 1

In [9]:
def dca_return(ending_index, dca_length):
    portfolio_equity = 0
    if ending_index < dca_length:
        return np.nan
    for i in range(ending_index - dca_length, ending_index):
        portfolio_equity = portfolio_equity + 1/dca_length
        portfolio_equity = portfolio_equity * (1 + df['1m_cumulative'].iloc[i+1])
    return portfolio_equity - 1

In [10]:
for period, duration in zip(periods, durations):
    df[f'{period}_dca_cumulative'] = 0
    for ending_index in df.index:
        df.loc[ending_index, f'{period}_dca_cumulative'] = dca_return(ending_index, duration)

In [11]:
for period, duration in zip(periods, durations):
    df[f'{period}_dca_annualized'] = (1 + df[f'{period}_dca_cumulative'])**(12/duration) - 1

In [12]:
for period, duration in zip(periods, durations):
    df[f'{period}_cumulative_difference'] = df[f'{period}_cumulative'] - df[f'{period}_dca_cumulative']

In [13]:
for period, duration in zip(periods, durations):
    df[f'{period}_difference_in_annualized'] = df[f'{period}_annualized'] - df[f'{period}_dca_annualized']

In [15]:
df.head(20)

,date,price,1m_cumulative,3m_cumulative,6m_cumulative,1y_cumulative,2y_cumulative,3y_cumulative,5y_cumulative,10y_cumulative,20y_cumulative,1m_annualized,3m_annualized,6m_annualized,1y_annualized,2y_annualized,3y_annualized,5y_annualized,10y_annualized,20y_annualized,1m_dca_cumulative,3m_dca_cumulative,6m_dca_cumulative,1y_dca_cumulative,2y_dca_cumulative,3y_dca_cumulative,5y_dca_cumulative,10y_dca_cumulative,20y_dca_cumulative,1m_dca_annualized,3m_dca_annualized,6m_dca_annualized,1y_dca_annualized,2y_dca_annualized,3y_dca_annualized,5y_dca_annualized,10y_dca_annualized,20y_dca_annualized,1m_cumulative_difference,3m_cumulative_difference,6m_cumulative_difference,1y_cumulative_difference,2y_cumulative_difference,3y_cumulative_difference,5y_cumulative_difference,10y_cumulative_difference,20y_cumulative_difference,1m_difference_in_annualized,3m_difference_in_annualized,6m_difference_in_annualized,1y_difference_in_annualized,2y_difference_in_annualized,3y_difference_in_annualized,5y_difference_in_annualized,10y_difference_in_annualized,20y_difference_in_annualized
0,1969-12-31,100.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1970-01-30,94.455,-0.055450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.495688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.055450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.495688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1970-02-27,97.405,0.031232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.446357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.031232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.446357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1970-03-31,97.708,0.003111,-0.022920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.037974,-0.088576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003111,0.004877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.037974,0.019650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-0.027797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-0.108226,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1970-04-30,88.578,-0.093442,-0.062220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.691859,-0.226601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.093442,-0.082094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.691859,-0.290109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.019874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.063508,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1970-05-29,82.990,-0.063086,-0.147990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.542492,-0.473040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.063086,-0.120569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.542492,-0.401856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-0.027421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-0.071184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1970-06-30,80.946,-0.024629,-0.171552,-0.190540,NaN,NaN,NaN,NaN,NaN,NaN,-0.258629,-0.528956,-0.344775,NaN,NaN,NaN,NaN,NaN,NaN,-0.024629,-0.094114,-0.130813,NaN,NaN,NaN,NaN,NaN,NaN,-0.258629,-0.326568,-0.244514,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-0.077438,-0.059727,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-0.202388,-0.100260,NaN,NaN,NaN,NaN,NaN,NaN
7,1970-07-31,85.965,0.062004,-0.029499,-0.089884,NaN,NaN,NaN,NaN,NaN,NaN,1.058331,-0.112878,-0.171689,NaN,NaN,NaN,NaN,NaN,NaN,0.062004,0.022784,-0.043194,NaN,NaN,NaN,NaN,NaN,NaN,1.058331,0.094299,-0.084522,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-0.052284,-0.046690,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-0.207177,-0.087167,NaN,NaN,NaN,NaN,NaN,NaN
8,1970-08-31,88.799,0.032967,0.069996,-0.088353,NaN,NaN,NaN,NaN,NaN,NaN,0.475832,0.310778,-0.168899,NaN,NaN,NaN,NaN,NaN,NaN,0.032967,0.066660,0.003823,NaN,NaN,NaN,NaN,NaN,NaN,0.475832,0.294504,0.007662,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.003337,-0.092176,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.016275,-0.176561,NaN,NaN,NaN,NaN,NaN,NaN
9,1970-09-30,91.667,0.032298,0.132446,-0.061827,NaN,NaN,NaN,NaN,NaN,NaN,0.464399,0.644639,-0.119832,NaN,NaN,NaN,NaN,NaN,NaN,0.032298,0.077024,0.051446,NaN,NaN,NaN,NaN,NaN,NaN,0.464399,0.345557,0.105538,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.055422,-0.113273,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.299081,-0.225370,NaN,NaN,NaN,NaN,

In [16]:
df.describe()

,price,1m_cumulative,3m_cumulative,6m_cumulative,1y_cumulative,2y_cumulative,3y_cumulative,5y_cumulative,10y_cumulative,20y_cumulative,1m_annualized,3m_annualized,6m_annualized,1y_annualized,2y_annualized,3y_annualized,5y_annualized,10y_annualized,20y_annualized,1m_dca_cumulative,3m_dca_cumulative,6m_dca_cumulative,1y_dca_cumulative,2y_dca_cumulative,3y_dca_cumulative,5y_dca_cumulative,10y_dca_cumulative,20y_dca_cumulative,1m_dca_annualized,3m_dca_annualized,6m_dca_annualized,1y_dca_annualized,2y_dca_annualized,3y_dca_annualized,5y_dca_annualized,10y_dca_annualized,20y_dca_annualized,1m_cumulative_difference,3m_cumulative_difference,6m_cumulative_difference,1y_cumulative_difference,2y_cumulative_difference,3y_cumulative_difference,5y_cumulative_difference,10y_cumulative_difference,20y_cumulative_difference,1m_difference_in_annualized,3m_difference_in_annualized,6m_difference_in_annualized,1y_difference_in_annualized,2y_difference_in_annualized,3y_difference_in_annualized,5y_difference_in_annualized,10y_difference_in_annualized,20y_difference_in_annualized
count,641.000000,640.000000,638.000000,635.000000,629.000000,617.000000,605.000000,581.000000,521.000000,401.000000,640.000000,638.000000,635.000000,629.000000,617.000000,605.000000,581.000000,521.000000,401.000000,640.000000,638.000000,635.000000,629.000000,617.000000,605.000000,581.000000,521.000000,401.000000,640.000000,638.000000,635.000000,629.000000,617.000000,605.000000,581.000000,521.000000,401.000000,640.0,638.000000,635.000000,629.000000,617.000000,605.000000,581.000000,521.000000,401.000000,640.0,638.000000,635.000000,629.000000,617.000000,605.000000,581.000000,521.000000,401.000000
mean,2177.928928,0.007950,0.024297,0.050096,0.103908,0.221399,0.342989,0.633083,1.674218,5.888702,0.229787,0.137624,0.115990,0.103908,0.098253,0.094491,0.092703,0.094257,0.092784,0.007950,0.016180,0.029082,0.055531,0.110575,0.167419,0.300240,0.730098,2.004305,0.229787,0.084717,0.064255,0.055531,0.051381,0.049777,0.050136,0.052592,0.052251,0.0,0.008117,0.021014,0.048376,0.110824,0.175571,0.332843,0.944120,3.884397,0.0,0.052907,0.051735,0.048376,0.046872,0.044714,0.042568,0.041666,0.040533
std,2255.920846,0.043111,0.077279,0.115367,0.171728,0.272065,0.364172,0.598426,1.181182,3.914582,0.613680,0.330275,0.240305,0.171728,0.123545,0.098524,0.074351,0.047054,0.031623,0.043111,0.055202,0.072483,0.100674,0.149270,0.191439,0.283726,0.515604,1.264482,0.613680,0.224095,0.146308,0.100674,0.071974,0.058006,0.044380,0.029184,0.021682,0.0,0.033626,0.058320,0.095910,0.161051,0.221290,0.373382,0.764111,2.849950,0.0,0.153412,0.125983,0.095910,0.071054,0.057307,0.042764,0.026018,0.016249
min,80.946000,-0.189601,-0.332204,-0.435526,-0.471197,-0.474002,-0.456299,-0.253635,-0.227533,0.884585,-0.919761,-0.801127,-0.681369,-0.471197,-0.274743,-0.183818,-0.056829,-0.025486,0.032193,-0.189601,-0.257202,-0.301863,-0.356104,-0.431389,-0.429834,-0.370301,-0.273522,0.138754,-0.919761,-0.695573,-0.512605,-0.356104,-0.245937,-0.170785,-0.088353,-0.031450,0.006518,0.0,-0.144280,-0.205963,-0.341797,-0.320562,-0.340030,-0.323285,-0.200129,0.103990,0.0,-0.378534,-0.326231,-0.341797,-0.170335,-0.118299,-0.063706,-0.018452,0.002925
25%,259.050000,-0.016661,-0.013196,-0.014298,0.006793,0.081242,0.157271,0.205998,0.876256,2.537849,-0.182593,-0.051749,-0.028392,0.006793,0.039828,0.049893,0.038172,0.064950,0.065214,-0.016661,-0.010684,-0.005265,0.003332,0.034940,0.062002,0.122059,0.392570,0.986294,-0.182593,-0.042057,-0.010503,0.003332,0.017320,0.020254,0.023300,0.033670,0.034909,0.0,-0.008910,-0.011200,-0.003994,0.020920,0.055379,0.115216,0.383921,1.660246,0.0,-0.035995,-0.021903,-0.003994,0.011457,0.017595,0.020124,0.024290,0.029566
50%,1503.506000,0.011799,0.027400,0.053283,0.120567,0.241454,0.319956,0.567995,1.335702,4.721031,0.151143,0.114188,0.109405,0.120567,0.114206,0.096949,0.094130,0.088533,0.091123,0.011799,0.020379,0.033002,0.066852,0.129932,0.180281,0.273792,0.621565,1.387957,0.151143,0.084041,0.067093,0.066852,0.

In [19]:
go.Figure(
    [
        go.Scatter(
            x=df['date'],
            y=df[column],
            name=column,
            mode='lines'
            )
        for column in ['5y_annualized', '5y_dca_annualized']
    ]
)

In [20]:
go.Figure(
    [
        go.Scatter(
            x=df['date'],
            y=df[column],
            name=column,
            mode='lines'
            )
        for column in ['10y_annualized', '10y_dca_annualized']
    ]
)

In [21]:
go.Figure(
    [
        go.Scatter(
            x=df['date'],
            y=df[column],
            name=column,
            mode='lines'
            )
        for column in ['20y_annualized', '20y_dca_annualized']
    ]
)

In [25]:
go.Figure(
    [
        go.Box(
            x=df[column],
            name=column,
            opacity=0.75
            )
        for column in df.columns[47:]
    ]
)